#문화데이터경진대회_냥냥펀치_영화 데이터

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote
import json
import time
import datetime
from tqdm import tqdm

In [ ]:
# '영화진흥위원회'-'영화관입장권통합전산망'-'역대 박스오피스' 데이터 불러오기

boxoffice = pd.read_excel('/content/drive/MyDrive/2023/deep daiv. spring/bookrec/KOBIS_역대_박스오피스_내역(공식통계_기준)_2023-06-26.xls.xlsx')
boxoffice

,순위,영화명,개봉일,매출액,관객수,Unnamed: 5,스크린수,Unnamed: 7,대표국적,국적,배급사
0,NaN,NaN,NaT,NaN,(S:서울 기준),NaN,(S:서울 기준),NaN,NaN,NaN,NaN
1,1.0,명량,2014-07-30,1.357484e+11,NaN,17613682.0,NaN,1587.0,한국,한국,(주)씨제이이엔엠
2,2.0,극한직업,2019-01-23,1.396480e+11,NaN,16264944.0,NaN,1978.0,한국,한국,(주)씨제이이엔엠
3,3.0,신과함께-죄와 벌,2017-12-20,1.156987e+11,NaN,14410754.0,NaN,1912.0,한국,한국,롯데쇼핑㈜롯데엔터테인먼트
4,4.0,국제시장,2014-12-17,1.109135e+11,NaN,14257115.0,NaN,966.0,한국,한국,(주)씨제이이엔엠
...,...,...,...,...,...,...,...,...,...,...,...
496,496.0,오늘의 연애,2015-01-14,1.492465e+10,NaN,1891993.0,NaN,709.0,한국,한국,(주)씨제이이엔엠
497,497.0,글러브,2011-01-20,1.375757e+10,NaN,1888100.0,NaN,605.0,한국,한국,CJ ENM
498,498.0,러브 액츄얼리,2003-12-05,0.000000e+00,NaN,1887000.0,S,41.0,미국,"미국,영국","(주)제이앤씨미디어그룹,와이드 릴리즈(주),UIP코리아,(유)조이앤시네마"
499,499.0,라디오 스타,2006-09-28,0.000000e+00,NaN,1879501.0,S,72.0,한국,한국,(주)시네마서비스


In [ ]:
boxoffice.drop(0, axis=0, inplace=True)

In [ ]:
boxoffice.reset_index(inplace=True)

In [ ]:
boxoffice.rename(columns = {"관객수": "Unnamed: 5", "Unnamed: 5":"관객수", "스크린수": "Unnamed: 7", "Unnamed: 7": "스크린수"}, inplace = True)

In [ ]:
boxoffice.drop(columns=['index', 'Unnamed: 5', 'Unnamed: 7'],axis=1, inplace=True)

In [ ]:
boxoffice['개봉일'] = boxoffice['개봉일'].apply(lambda x: str(x)[:10])

In [ ]:
for i in range(boxoffice.shape[0]):
  boxoffice.loc[i,'개봉일'] = boxoffice.loc[i,'개봉일'].replace('-','')

In [ ]:
#'한국영상자료원'-'KMDb(한국영화데이터베이스)'-'Open API
CLIENT_ID=''

def search_movie(query1, query2):
    request = Request('http://api.koreafilm.or.kr/openapi-data2/wisenut/search_api/search_json2.jsp?collection=kmdb_new2&query=' + quote(query1) +'&releaseDts='+quote(query2)+'&releaseDte='+quote(query2) +'&ServiceKey=' + CLIENT_ID)
    request.add_header('X-KMDb-Client-Id', CLIENT_ID)


    response = urlopen(request).read().decode('utf-8')
    search_result = json.loads(response)

    return search_result

In [ ]:
boxoffice['줄거리']=0
boxoffice['장르']=0
boxoffice['키워드']=0
boxoffice['감독']=0

In [ ]:
boxoffice

,순위,영화명,개봉일,매출액,관객수,스크린수,대표국적,국적,배급사,줄거리,장르,키워드,감독
0,1.0,명량,20140730,1.357484e+11,17613682.0,1587.0,한국,한국,(주)씨제이이엔엠,0,0,0,0
1,2.0,극한직업,20190123,1.396480e+11,16264944.0,1978.0,한국,한국,(주)씨제이이엔엠,0,0,0,0
2,3.0,신과함께-죄와 벌,20171220,1.156987e+11,14410754.0,1912.0,한국,한국,롯데쇼핑㈜롯데엔터테인먼트,0,0,0,0
3,4.0,국제시장,20141217,1.109135e+11,14257115.0,966.0,한국,한국,(주)씨제이이엔엠,0,0,0,0
4,5.0,어벤져스: 엔드게임,20190424,1.221827e+11,13934592.0,2835.0,미국,미국,월트디즈니컴퍼니코리아 유한책임회사,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496.0,오늘의 연애,20150114,1.492465e+10,1891993.0,709.0,한국,한국,(주)씨제이이엔엠,0,0,0,0
496,497.0,글러브,20110120,1.375757e+10,1888100.0,605.0,한국,한국,CJ ENM,0,0,0,0
497,498.0,러브 액츄얼리,20031205,0.000000e+00,1887000.0,41.0,미국,"미국,영국","(주)제이앤씨미디어그룹,와이드 릴리즈(주),UIP코리아,(유)조이앤시네마",0,0,0,0
498,499.0,라디오 스타,20060928,0.000000e+00,1879501.0,72.0,한국,한국,(주)시네마서비스,0,0,0,0


In [ ]:
for i in tqdm(range(boxoffice.shape[0])):
  if __name__ == "__main__":
    try:
      movies = search_movie(boxoffice.loc[i,'영화명'],str(boxoffice.loc[i,'개봉일']))['Data'][0]['Result'][0]
      boxoffice.loc[i,'줄거리'] = movies['plots']['plot'][0]['plotText']
      boxoffice.loc[i,'장르'] = movies['genre']
      boxoffice.loc[i,'키워드'] = movies['keywords']
      boxoffice.loc[i,'감독'] = movies['directors']['director'][0]['directorNm']

    except:
      continue

boxoffice

100%|██████████| 500/500 [04:36<00:00,  1.81it/s]


,순위,영화명,개봉일,매출액,관객수,스크린수,대표국적,국적,배급사,줄거리,장르,키워드,감독
0,1.0,명량,20140730,1.357484e+11,17613682.0,1587.0,한국,한국,(주)씨제이이엔엠,"1597년 임진왜란 6년, 오랜 전쟁으로 인해 혼란이 극에 달한 조선. 무서운 속도...","시대극/사극,액션,전기","사극,수군,실화바탕,실화바탕,역사물,위인,이순신,임진왜란,전쟁,해전,명량해전,명량대...",김한민
1,2.0,극한직업,20190123,1.396480e+11,16264944.0,1978.0,한국,한국,(주)씨제이이엔엠,낮에는 치킨장사! 밤에는 잠복근무!지금까지 이런 수사는 없었다!불철주야 달리고 구르...,코메디,"잠복근무,마약단속반,위장경찰,위장취업,형사,경찰,마약범죄단,마약밀매업자,마약상,맛집...",이병헌
2,3.0,신과함께-죄와 벌,20171220,1.156987e+11,14410754.0,1912.0,한국,한국,롯데쇼핑㈜롯데엔터테인먼트,"저승 법에 의하면, 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다.살인...","드라마,판타지","웹툰원작,저승,저승사자,검사,사후세계,시리즈,소방관,염라대왕",김용화
3,4.0,국제시장,20141217,1.109135e+11,14257115.0,966.0,한국,한국,(주)씨제이이엔엠,가장 평범한 아버지의 가장 위대한 이야기’1950년대 한국전쟁 이후로부터 현재에 이...,드라마,"6.25,6.25,625,현대사,한국전쟁,부산,부산광역시",윤제균
4,5.0,어벤져스: 엔드게임,20190424,1.221827e+11,13934592.0,2835.0,미국,미국,월트디즈니컴퍼니코리아 유한책임회사,인피니티 워 이후 절반만 살아남은 지구 마지막 희망이 된 어벤져스 먼저 떠난 그들을...,"액션,SF","마블코믹스,슈퍼히어로,다크히어로,어벤져스,아이언맨,캡틴아메리카,블랙위도우,토르,헐크...",안소니 루소
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496.0,오늘의 연애,20150114,1.492465e+10,1891993.0,709.0,한국,한국,(주)씨제이이엔엠,해 달라는 대로 다 해주는데 100일도 못 가 항상 여자친구에게 차이는 초등학교 교...,"드라마,멜로드라마","사랑,연애,친구,소꿉친구,기상캐스터,썸",박진표
496,497.0,글러브,20110120,1.375757e+10,1888100.0,605.0,한국,한국,CJ ENM,퇴출직전의 꼴통 프로 투수와 말도 안 통하고(?) 꽉 막힌 만년꼴찌 야구부원들의 1...,"드라마,스포츠","고교야구,도전,스포츠,실화바탕,실화바탕,야구,야구감독,야구부,야구선수,야구스타,전국...",강우석
497,498.0,러브 액츄얼리,20031205,0.000000e+00,1887000.0,41.0,미국,"미국,영국","(주)제이앤씨미디어그룹,와이드 릴리즈(주),UIP코리아,(유)조이앤시네마",새로 부임한 매력적인 미혼의 영국 수상(휴 그랜트)은 발랄하고 귀여운 비서 나탈리(...,"코메디,드라마,멜로/로맨스","결혼,공항,로맨스,수상,짝사랑,크리스마스,재개봉,옴니버스형식",리차드 커티스
498,499.0,라디오 스타,20060928,0.000000e+00,1879501.0,72.0,한국,한국,(주)시네마서비스,"명곡 비와 당신으로 88년 가수 왕을 차지했던 최곤은 그 후 대마초 사건, 폭행사건...","코메디,드라마","대마초,인기가수,통기타,매니저,라디오,라디오DJ,라디오쇼,라디오PD,다방,다방레지,...",이준익


In [ ]:
#개봉일자가 하루씩 앞이거나 뒤인 경우가 있어 수기로 조정하여 검색(개봉일자가 데이터보다 하루 전)
for i in tqdm(range(boxoffice.shape[0])):
  if __name__ == "__main__":
    if boxoffice.loc[i,'줄거리']==0:
      try:
        day=datetime.datetime(int(str(boxoffice.loc[i,'개봉일'])[:4]),int(str(boxoffice.loc[i,'개봉일'])[4:6]),int(str(boxoffice.loc[i,'개봉일'])[6:]))
        day_before=day - datetime.timedelta(days=1)
        day_before = day_before.strftime('%Y%m%d')
        movies = search_movie(boxoffice.loc[i,'영화명'],day_before)['Data'][0]['Result'][0]
        boxoffice.loc[i,'줄거리'] = movies['plots']['plot'][0]['plotText']
        boxoffice.loc[i,'장르'] = movies['genre']
        boxoffice.loc[i,'키워드'] = movies['keywords']
        boxoffice.loc[i,'감독'] = movies['directors']['director'][0]['directorNm']
      except:
        continue

boxoffice

100%|██████████| 500/500 [00:14<00:00, 35.54it/s]


,순위,영화명,개봉일,매출액,관객수,스크린수,대표국적,국적,배급사,줄거리,장르,키워드,감독
0,1.0,명량,20140730,1.357484e+11,17613682.0,1587.0,한국,한국,(주)씨제이이엔엠,"1597년 임진왜란 6년, 오랜 전쟁으로 인해 혼란이 극에 달한 조선. 무서운 속도...","시대극/사극,액션,전기","사극,수군,실화바탕,실화바탕,역사물,위인,이순신,임진왜란,전쟁,해전,명량해전,명량대...",김한민
1,2.0,극한직업,20190123,1.396480e+11,16264944.0,1978.0,한국,한국,(주)씨제이이엔엠,낮에는 치킨장사! 밤에는 잠복근무!지금까지 이런 수사는 없었다!불철주야 달리고 구르...,코메디,"잠복근무,마약단속반,위장경찰,위장취업,형사,경찰,마약범죄단,마약밀매업자,마약상,맛집...",이병헌
2,3.0,신과함께-죄와 벌,20171220,1.156987e+11,14410754.0,1912.0,한국,한국,롯데쇼핑㈜롯데엔터테인먼트,"저승 법에 의하면, 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다.살인...","드라마,판타지","웹툰원작,저승,저승사자,검사,사후세계,시리즈,소방관,염라대왕",김용화
3,4.0,국제시장,20141217,1.109135e+11,14257115.0,966.0,한국,한국,(주)씨제이이엔엠,가장 평범한 아버지의 가장 위대한 이야기’1950년대 한국전쟁 이후로부터 현재에 이...,드라마,"6.25,6.25,625,현대사,한국전쟁,부산,부산광역시",윤제균
4,5.0,어벤져스: 엔드게임,20190424,1.221827e+11,13934592.0,2835.0,미국,미국,월트디즈니컴퍼니코리아 유한책임회사,인피니티 워 이후 절반만 살아남은 지구 마지막 희망이 된 어벤져스 먼저 떠난 그들을...,"액션,SF","마블코믹스,슈퍼히어로,다크히어로,어벤져스,아이언맨,캡틴아메리카,블랙위도우,토르,헐크...",안소니 루소
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496.0,오늘의 연애,20150114,1.492465e+10,1891993.0,709.0,한국,한국,(주)씨제이이엔엠,해 달라는 대로 다 해주는데 100일도 못 가 항상 여자친구에게 차이는 초등학교 교...,"드라마,멜로드라마","사랑,연애,친구,소꿉친구,기상캐스터,썸",박진표
496,497.0,글러브,20110120,1.375757e+10,1888100.0,605.0,한국,한국,CJ ENM,퇴출직전의 꼴통 프로 투수와 말도 안 통하고(?) 꽉 막힌 만년꼴찌 야구부원들의 1...,"드라마,스포츠","고교야구,도전,스포츠,실화바탕,실화바탕,야구,야구감독,야구부,야구선수,야구스타,전국...",강우석
497,498.0,러브 액츄얼리,20031205,0.000000e+00,1887000.0,41.0,미국,"미국,영국","(주)제이앤씨미디어그룹,와이드 릴리즈(주),UIP코리아,(유)조이앤시네마",새로 부임한 매력적인 미혼의 영국 수상(휴 그랜트)은 발랄하고 귀여운 비서 나탈리(...,"코메디,드라마,멜로/로맨스","결혼,공항,로맨스,수상,짝사랑,크리스마스,재개봉,옴니버스형식",리차드 커티스
498,499.0,라디오 스타,20060928,0.000000e+00,1879501.0,72.0,한국,한국,(주)시네마서비스,"명곡 비와 당신으로 88년 가수 왕을 차지했던 최곤은 그 후 대마초 사건, 폭행사건...","코메디,드라마","대마초,인기가수,통기타,매니저,라디오,라디오DJ,라디오쇼,라디오PD,다방,다방레지,...",이준익


In [ ]:
#개봉일자가 하루씩 앞이거나 뒤인 경우가 있어 수기로 조정하여 검색(개봉일자가 데이터보다 하루 후)
for i in tqdm(range(boxoffice.shape[0])):
  if __name__ == "__main__":
    if boxoffice.loc[i,'줄거리']==0:
      try:
        day=datetime.datetime(int(str(boxoffice.loc[i,'개봉일'])[:4]),int(str(boxoffice.loc[i,'개봉일'])[4:6]),int(str(boxoffice.loc[i,'개봉일'])[6:]))
        day_before=day - datetime.timedelta(days=-1)
        day_before = day_before.strftime('%Y%m%d')
        movies = search_movie(boxoffice.loc[i,'영화명'],day_before)['Data'][0]['Result'][0]
        boxoffice.loc[i,'줄거리'] = movies['plots']['plot'][0]['plotText']
        boxoffice.loc[i,'장르'] = movies['genre']
        boxoffice.loc[i,'키워드'] = movies['keywords']
        boxoffice.loc[i,'감독'] = movies['directors']['director'][0]['directorNm']
      except:
        continue

boxoffice

100%|██████████| 500/500 [00:23<00:00, 21.65it/s]


,순위,영화명,개봉일,매출액,관객수,스크린수,대표국적,국적,배급사,줄거리,장르,키워드,감독
0,1.0,명량,20140730,1.357484e+11,17613682.0,1587.0,한국,한국,(주)씨제이이엔엠,"1597년 임진왜란 6년, 오랜 전쟁으로 인해 혼란이 극에 달한 조선. 무서운 속도...","시대극/사극,액션,전기","사극,수군,실화바탕,실화바탕,역사물,위인,이순신,임진왜란,전쟁,해전,명량해전,명량대...",김한민
1,2.0,극한직업,20190123,1.396480e+11,16264944.0,1978.0,한국,한국,(주)씨제이이엔엠,낮에는 치킨장사! 밤에는 잠복근무!지금까지 이런 수사는 없었다!불철주야 달리고 구르...,코메디,"잠복근무,마약단속반,위장경찰,위장취업,형사,경찰,마약범죄단,마약밀매업자,마약상,맛집...",이병헌
2,3.0,신과함께-죄와 벌,20171220,1.156987e+11,14410754.0,1912.0,한국,한국,롯데쇼핑㈜롯데엔터테인먼트,"저승 법에 의하면, 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다.살인...","드라마,판타지","웹툰원작,저승,저승사자,검사,사후세계,시리즈,소방관,염라대왕",김용화
3,4.0,국제시장,20141217,1.109135e+11,14257115.0,966.0,한국,한국,(주)씨제이이엔엠,가장 평범한 아버지의 가장 위대한 이야기’1950년대 한국전쟁 이후로부터 현재에 이...,드라마,"6.25,6.25,625,현대사,한국전쟁,부산,부산광역시",윤제균
4,5.0,어벤져스: 엔드게임,20190424,1.221827e+11,13934592.0,2835.0,미국,미국,월트디즈니컴퍼니코리아 유한책임회사,인피니티 워 이후 절반만 살아남은 지구 마지막 희망이 된 어벤져스 먼저 떠난 그들을...,"액션,SF","마블코믹스,슈퍼히어로,다크히어로,어벤져스,아이언맨,캡틴아메리카,블랙위도우,토르,헐크...",안소니 루소
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496.0,오늘의 연애,20150114,1.492465e+10,1891993.0,709.0,한국,한국,(주)씨제이이엔엠,해 달라는 대로 다 해주는데 100일도 못 가 항상 여자친구에게 차이는 초등학교 교...,"드라마,멜로드라마","사랑,연애,친구,소꿉친구,기상캐스터,썸",박진표
496,497.0,글러브,20110120,1.375757e+10,1888100.0,605.0,한국,한국,CJ ENM,퇴출직전의 꼴통 프로 투수와 말도 안 통하고(?) 꽉 막힌 만년꼴찌 야구부원들의 1...,"드라마,스포츠","고교야구,도전,스포츠,실화바탕,실화바탕,야구,야구감독,야구부,야구선수,야구스타,전국...",강우석
497,498.0,러브 액츄얼리,20031205,0.000000e+00,1887000.0,41.0,미국,"미국,영국","(주)제이앤씨미디어그룹,와이드 릴리즈(주),UIP코리아,(유)조이앤시네마",새로 부임한 매력적인 미혼의 영국 수상(휴 그랜트)은 발랄하고 귀여운 비서 나탈리(...,"코메디,드라마,멜로/로맨스","결혼,공항,로맨스,수상,짝사랑,크리스마스,재개봉,옴니버스형식",리차드 커티스
498,499.0,라디오 스타,20060928,0.000000e+00,1879501.0,72.0,한국,한국,(주)시네마서비스,"명곡 비와 당신으로 88년 가수 왕을 차지했던 최곤은 그 후 대마초 사건, 폭행사건...","코메디,드라마","대마초,인기가수,통기타,매니저,라디오,라디오DJ,라디오쇼,라디오PD,다방,다방레지,...",이준익


In [ ]:
boxoffice.loc[boxoffice['줄거리']==0]

,순위,영화명,개봉일,매출액,관객수,스크린수,대표국적,국적,배급사,줄거리,장르,키워드,감독
58,59.0,1987,20171227,5.816558e+10,7231638.0,1299.0,한국,한국,(주)씨제이이엔엠,0,0,0,0
67,68.0,은밀하게 위대하게,20130605,4.870089e+10,6959083.0,1341.0,한국,한국,(주)쇼박스,0,0,0,0
85,86.0,인셉션,20100721,4.410655e+10,5926948.0,528.0,미국,미국,"(주)디스테이션,워너브러더스 코리아(주)",0,0,0,0
101,102.0,2012,20091112,3.941517e+10,5462953.0,789.0,미국,미국,한국소니픽쳐스릴리징브에나비스타영화㈜,0,0,0,0
147,148.0,박물관이 살아있다!,20061221,0.000000e+00,4612831.0,86.0,미국,미국,이십세기폭스코리아(주),0,0,0,0
152,153.0,맘마미아!,20080903,2.957078e+10,4554785.0,457.0,미국,미국,유니버설픽쳐스인터내셔널 코리아(유),0,0,0,0
160,161.0,엑스맨: 데이즈 오브 퓨처 패스트,20140522,3.465456e+10,4313871.0,1056.0,미국,미국,이십세기폭스코리아(주),0,0,0,0
245,246.0,탐정: 리턴즈,20180613,2.690946e+10,3152872.0,1041.0,한국,한국,(주)씨제이이엔엠,0,0,0,0
247,248.0,오! 브라더스,20030904,0.000000e+00,3148748.0,51.0,한국,한국,(주)쇼박스,0,0,0,0
268,269.0,하울의 움직이는 성,20041223,0.000000e+00,3015165.0,65.0,일본,일본,"(주)이수C&E,(주)시네마서비스",0,0,0,0


In [ ]:
boxoffice.to_excel('/content/drive/MyDrive/2023/deep daiv. spring/bookrec/역대 한국 박스오피스 흥행작_장르, 키워드, 감독 포함.xlsx',index=False)

개봉일자가 '영화진흥위원회'-'영화관입장권통합전산망'-'역대 박스오피스'의 데이터와 맞지 않아 검색이 되지 않은 영화는 약 50개가 있었습니다.

그중 일부는 개봉일자를 하루 전, 하루 후로 검색하여 채워넣었습니다. 이에 최종적으로 줄거리가 0인 데이터는 18개가 나왔습니다. 18개의 영화 중 15개는 KMDb 웹사이트에서 수기로 검색하여 줄거리, 장르, 키워드, 감독을 넣었으며 검색이 안 되는 3개의 영화는 drop 하였습니다. 따라서 총 497개의 영화 데이터를 준비하였습니다.